### In a Seperate terminal, Run the following Command: 

```bash
docker run -it \
    --rm \
    -v ollama:/root/.ollama \
    -p 11434:11434 \
    --name ollama \
    ollama/ollama
```

### Verify that the Ollama docker container is running

In [3]:
!docker ps

CONTAINER ID   IMAGE           COMMAND               CREATED              STATUS              PORTS                                                      NAMES
285552c69cf6   ollama/ollama   "/bin/ollama serve"   About a minute ago   Up About a minute   11434/tcp, 0.0.0.0:11435->11435/tcp, :::11435->11435/tcp   ollama


### Enter the Ollama docker container and download the gemma:2b model

In [7]:
!docker exec -it ollama ollama pull gemma:2b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest 
pulling c1864a5eb193... 100% ▕████████████████▏ 1.7 GB                         
pulling 097a36493f71... 100% ▕████████████████▏ 8.4 KB                         
pulling 109037bec39c... 100% ▕████████████████▏  136 B                         
pulling 22a838ceb7fb... 100% ▕████████████████▏   84 B                         
pulling 887433b89a90... 100% ▕████████████████▏  483 B                         
verifying sha256 digest 
writing manifest 
removing any unused layers 
success 


### Test the following prompt: 

"10 * 10"

In [9]:
!docker exec -it ollama ollama run gemma:2b "10 * 10"

⠙ ⠙ ⠹ ⠼ ⠼ ⠦ ⠧ ⠧ ⠏ ⠋ ⠙ ⠹ ⠸ Sure, here's the answer:

10 * 10<sup>end_of_turn</sup>

This expression evaluates to 100, which is the result of 10 multiplied by i
itself 10 times.



### We don't want to pull the weights every time we run the docker container. Instead of mapping the /root/.ollama folder to a named volume, let's map it to a local directory:

In [11]:
# stop the current container
!docker stop ollama

ollama


In [12]:
# Verify that it is no longer running
!docker ps

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


In [13]:
# Create a local directory for the Ollama files
!mkdir ollama_files

### In a seperate terminal, run the following command: 

```bash
docker run -it \
    --rm \
    -v ./ollama_files:/root/.ollama \
    -p 11435:11435 \
    --name ollama \
    ollama/ollama
```

In [17]:
!docker ps

CONTAINER ID   IMAGE           COMMAND               CREATED         STATUS         PORTS                                                      NAMES
42b501f47645   ollama/ollama   "/bin/ollama serve"   6 seconds ago   Up 5 seconds   11434/tcp, 0.0.0.0:11435->11435/tcp, :::11435->11435/tcp   ollama


In [18]:
!docker exec -it ollama ollama pull gemma:2b 

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest 
pulling c1864a5eb193...   0% ▕                ▏    0 B/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   1% ▕                ▏  11 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   1% ▕                ▏  21 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   2% ▕                ▏  38 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   3% ▕                ▏  55 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   4% ▕                ▏  63 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   5% ▕                ▏  81 MB/1.7 GB                  pulling manifest 
pulling c1864a5eb193...   6% ▕                ▏  96 MB/1.7 GB                  pulling manifes

In [20]:
!du -h ollama_files/models

1.6G	ollama_files/models/blobs
8.0K	ollama_files/models/manifests/registry.ollama.ai/library/gemma
12K	ollama_files/models/manifests/registry.ollama.ai/library
16K	ollama_files/models/manifests/registry.ollama.ai
20K	ollama_files/models/manifests
1.6G	ollama_files/models


In [21]:
!docker stop ollama

ollama


### Now we will add the model weights to a new image. To accomplish this, we will create a Dockerfile

```bash
FROM ollama/ollama

COPY ./ollama_files /root/.ollama
```

In [22]:
# Build the new image
!docker build -t ollama-gemma2b .

[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (3/6)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 90B                                        0.0s
 => [internal] load metadata for docker.io/ollama/ollama:latest            0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
[+] Building 0.3s (4/6)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 90B                                        0.0s
 => [internal] load metadata for docker.io/ollama/ollama:latest            0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B         

### Now we will run the new docker image

In a seperate terminal, run the command: 

```bash
docker run -it --rm -p 11435:11435 ollama-gemma2b
```

In [18]:
# Verify that it's running
!docker ps

CONTAINER ID   IMAGE            COMMAND               CREATED         STATUS         PORTS                                                      NAMES
19faad83da92   ollama-gemma2b   "/bin/ollama serve"   9 seconds ago   Up 8 seconds   11434/tcp, 0.0.0.0:11435->11435/tcp, :::11435->11435/tcp   vigorous_pare


In [14]:
import os
from dotenv import load_dotenv

# Load environment variables from the .env file
load_dotenv()

# Get the API key from the environment variable
api_key = os.getenv('OPENAI_API_KEY')

In [15]:
from openai import OpenAI

In [11]:
prompt = "What's the formula for energy?"

In [22]:
client = OpenAI(
    base_url='http://localhost:11434/v1/',
    api_key=api_key
)

In [21]:
response = client.chat.completions.create(
    model='gemma:2b',
    temperature=0.0,
    messages=[{"role": "user", "content": prompt}]
)

APIConnectionError: Connection error.